! pip install azure-search-documents --pre
! pip install openai
! pip install python-dotenv
! pip install --upgrade azure-search-documents
!pip install --upgrade langchain
! pip install pandasql

In [18]:
# Import required libraries  
import os  
import json  
import openai  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SemanticSettings,  
    VectorSearch, 
    HnswVectorSearchAlgorithmConfiguration,  
)  

from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import CharacterTextSplitter
from azure.storage.blob import BlobServiceClient
import tempfile
from langchain.document_loaders import PyPDFLoader
from azure.core.exceptions import HttpResponseError 
from langchain.embeddings.openai import OpenAIEmbeddings

In [19]:
# Configure environment variables  
#load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
credential = AzureKeyCredential("jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg")

In [20]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://testingchat.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "e8143eabb02541259054426630db12a7"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
model: str = "text-embedding-ada-002"
#pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"     

AZURE_SEARCH_SERVICE_ENDPOINT = "https://genaisafety.search.windows.net"
AZURE_SEARCH_ADMIN_KEY = "jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg"

In [21]:
def create_index(AZURE_SEARCH_INDEX_NAME, fields):
    vector_search = VectorSearch(
        algorithm_configurations=[
            HnswVectorSearchAlgorithmConfiguration(
                name="my-vector-config",
                kind="hnsw",
                parameters={
                    "m": 4,
                    "efConstruction": 400,
                    "efSearch": 500,
                    "metric": "cosine"
                }
            )
        ]
    )


    semantic_config = SemanticConfiguration(
        name="my-semantic-config",
        prioritized_fields=PrioritizedFields(
            title_field=SemanticField(field_name="id"),
            prioritized_keywords_fields=[SemanticField(field_name="content")]
        )
    )

    # Create the semantic settings with the configuration
    semantic_settings = SemanticSettings(configurations=[semantic_config])

    # Create the search index with the semantic settings
    index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME, fields=fields,
                        vector_search=vector_search, semantic_settings=semantic_settings)

    result = search_client.create_or_update_index(index)
    return result

In [22]:
embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 

# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Replace with your container name
container_name = "safety"

# Create a BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

In [23]:
blobs = container_client.list_blobs()
dict_data = []
for blob in blobs:
    if blob.name.startswith("etq-data"):
        #blob_client = container_client.get_blob_client(blob)
        pdf_file_name=str(blob.name).split("/")[1] 
        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        print(bid)

vw-etq-events-fast
vw-reportit-allreport-fast


In [24]:
import pandas as pd
import io
from azure.storage.blob import BlobServiceClient

# Replace with your actual connection string
azure_connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Replace with your container name
container_name = "safety"

try:
    # Connect to Azure Blob Storage
    blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
    container_client = blob_service_client.get_container_client(container_name)

    # List all blobs in the container
    blobs = container_client.list_blobs()

    # Create an empty list to store DataFrame objects
    dfs = {}

    # Loop through the blobs and process those that match the prefix
    for blob in blobs:
        if blob.name.startswith("etq-data"):
            # Extract file name and bid name
            file_name = str(blob.name).split("/")[1]
            bid_name = str(blob.name).split("/")[1].split(".")[0].lower().replace(" ", "_").replace("_", "-")

            # Get the blob client for reading the blob data
            blob_client = container_client.get_blob_client(blob.name)
            print(blob.name)

            # Download the blob data as a stream
            blob_data = blob_client.download_blob()

            # Read the stream as a string and then into a Pandas DataFrame
            blob_text = blob_data.readall().decode('utf-8')
            df = pd.read_csv(io.StringIO(blob_text))

            # Append the DataFrame to the list
            dfs[file_name] = df

    # Now you can work with the list of DataFrames (dfs) as needed
    print(dfs.keys())

except Exception as e:
    print(f'Error: {str(e)}')


etq-data/vw-ETQ-Events-fast
etq-data/vw_ReportIt_AllReport_fast
dict_keys(['vw-ETQ-Events-fast', 'vw_ReportIt_AllReport_fast'])


In [25]:
dfs['vw-ETQ-Events-fast'].head(1)

,Division,INCIDENT_ID,EtqNumber,EventDate,Airline,Flight,Tail,Risk,ReportType,IrropType,...,RiskNotes,CreatedDate,Likelihood,Severity,ErrStation,DiscStation,EventLevel,DAGSRB,InvestReq,Time
0,Inflight,205,204,2016-10-28,NaN,2094,428QX,1,Inflight : Other Event,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN


In [27]:
dfs['vw_ReportIt_AllReport_fast'].head(1)

,Carrier,EventType,SubType,Division,ID,ObjectId,WeekStart,SubmittedDate,Title,Risk,...,Description,Category,EventDate,Recommendations,Linked_eventNumber,Source,ErrorStation,DiscoveredStation,PSID,DISPLAY_NAME
0,AS,Report,Irreg,Inflight,49020,49051,2018-01-01,2018-01-01 00:29:25,Inflight : Catering/Service Interruption,NaN,...,Was only catered with one beverage cart on a d...,Meal Issue,2017-12-31 12:00:00,NaN,39303.0,Mobile Application,NaN,NaN,1050875,Alexandra Prewett


In [35]:
len(dfs['vw-ETQ-Events-fast']['EtqNumber'].unique())

296968

In [36]:
len(dfs['vw_ReportIt_AllReport_fast']['Linked_eventNumber'].unique())

295493

In [37]:
len(not_in_array2)

2083

In [38]:
# Find elements that are in array1 but not in array2
not_in_array2 = list(set(dfs['vw-ETQ-Events-fast']['EtqNumber'].unique()) - set(dfs['vw_ReportIt_AllReport_fast']['Linked_eventNumber'].unique()))

# Find elements that are in array2 but not in array1
not_in_array1 = list(set(dfs['vw_ReportIt_AllReport_fast']['Linked_eventNumber'].unique()) - set(dfs['vw-ETQ-Events-fast']['EtqNumber'].unique()))

print("Elements in array1 but not in array2:", not_in_array2)
print("Elements in array2 but not in array1:", not_in_array1)

Elements in array1 but not in array2: [212993, 2, 147464, 213004, 147471, 180241, 278547, 40983, 81948, 106526, 32807, 8238, 90194, 163923, 180306, 114779, 114780, 188508, 139358, 270427, 106593, 106594, 147558, 114791, 131182, 172142, 106624, 155779, 90245, 90247, 90248, 90249, 8330, 131209, 131210, 16525, 114830, 16527, 114831, 114832, 131213, 172177, 278672, 106652, 90270, 163999, 114850, 164002, 164004, 254116, 297998, 106667, 155851, 204, 32974, 262355, 139476, 32987, 311516, 98528, 229606, 254187, 123117, 336117, 106763, 155931, 262429, 123169, 262440, 237880, 123194, 90428, 131392, 131401, 131402, 205135, 131413, 106842, 270691, 106853, 205157, 16746, 205164, 254321, 221556, 33154, 123267, 131460, 156039, 131464, 41359, 41360, 98703, 319889, 41368, 123290, 90526, 98721, 131490, 98724, 336292, 188839, 237992, 197042, 98741, 164291, 131524, 278981, 205260, 115153, 213467, 147941, 486, 106986, 49645, 180723, 328182, 172545, 123396, 123397, 131588, 131591, 123407, 123409, 98846, 988

In [10]:
# 1. Perform a left join between etq_df and ri_df
merged_df = pd.merge(dfs['vw-ETQ-Events-fast'], dfs['vw_ReportIt_AllReport_fast'], left_on='EtqNumber', right_on='Linked_eventNumber', how='left')
merged_df.head(1)

,Division_x,INCIDENT_ID,EtqNumber,EventDate_x,Airline,Flight_x,Tail_x,Risk_x,ReportType,IrropType,...,Description_y,Category,EventDate_y,Recommendations,Linked_eventNumber,Source,ErrorStation,DiscoveredStation,PSID,DISPLAY_NAME
0,Inflight,205,204,2016-10-28,NaN,2094,428QX,1,Inflight : Other Event,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
merged_df.shape

(356440, 53)

In [12]:
merged_df.columns

Index(['Division_x', 'INCIDENT_ID', 'EtqNumber', 'EventDate_x', 'Airline',
       'Flight_x', 'Tail_x', 'Risk_x', 'ReportType', 'IrropType', 'Cause',
       'CauseOther', 'Subject', 'Description_x', 'Origin', 'Destination',
       'Location_x', 'WSRDate', 'OPLEOI', 'AnalystNotes', 'RiskNotes',
       'CreatedDate', 'Likelihood', 'Severity', 'ErrStation', 'DiscStation',
       'EventLevel', 'DAGSRB', 'InvestReq', 'Time', 'Carrier', 'EventType',
       'SubType', 'Division_y', 'ID', 'ObjectId', 'WeekStart', 'SubmittedDate',
       'Title', 'Risk_y', 'Location_y', 'Tail_y', 'Flight_y', 'Description_y',
       'Category', 'EventDate_y', 'Recommendations', 'Linked_eventNumber',
       'Source', 'ErrorStation', 'DiscoveredStation', 'PSID', 'DISPLAY_NAME'],
      dtype='object')

In [13]:
# 2. Apply the filtering conditions
event_start_date = '2023-01-01'
event_end_date = '2023-02-01'

filtered_df = merged_df[(merged_df['EventDate_x'] >= event_start_date) &
                        (merged_df['EventDate_x'] < event_end_date) &
                        (merged_df['Recommendations'].notnull())]

# 3. Select the desired columns
selected_columns = ['ID','Division_x', 'EventDate_x', 'Subject', 'Description_x', 'Recommendations']
result_df = filtered_df[selected_columns]
result_df.head(1)

,ID,Division_x,EventDate_x,Subject,Description_x,Recommendations
191392,407297.0,Inflight,2023-01-15,Pax travelling with PETC; not compliant with p...,- Report # 407297\n \nWendy Boyer in 20C had h...,Give her a warning. If she does it again she ...


In [14]:
selected_columns = ['ID','Division_x', 'EventDate_x', 'Subject', 'Description_x', 'Recommendations']
result_df = merged_df[selected_columns]
df_concat = result_df.copy()
df_concat.head(2)


,ID,Division_x,EventDate_x,Subject,Description_x,Recommendations
0,NaN,Inflight,2016-10-28,NaN,Flight Attendant on board seemed to have a Sam...,NaN
1,NaN,NaN,2016-12-10,Irregularity Report - Pax reluctant to stow ba...,NaN,NaN


In [15]:
df_concat.shape

(356440, 6)

In [16]:
# Convert all columns to string type
df_concat = df_concat.astype(str)

# Concatenate all columns' data into a single column
df_concat['Concatenated'] = df_concat.apply(lambda row: ' '.join(row), axis=1)

In [39]:
# Embed the 'Concatenated' column
df_concat['content_vector'] = df_concat['Concatenated'].apply(lambda text: embeddings.embed_query(str(text)))

In [ ]:
df_concat['id_vector'] = df_concat['Concatenated'].apply(lambda text: embeddings.embed_query(str(text)))

In [ ]:
# Display the resulting DataFrame
df_concat.head(2)

In [ ]:
# Determine the total number of rows in the DataFrame
total_rows = len(df_concat)

# Calculate the number of rows in each half
rows_per_half = total_rows // 200

In [ ]:
--reference
# Create slices of the DataFrame for each half
dataframes_list = []
data_dict_list = []
partitionid = []
#ID = []
Content = []
Content_Vector = []
for i in range(200):
    start_row = i * rows_per_half
    end_row = (i + 1) * rows_per_half
    
    df_slice = df_concat.iloc[start_row:end_row] #slice df_concat
    #dataframes_list.append(df_slice)
    #convert DF to JSON
    partitionid.append(i)
    for index in df_slice['ID'].index:
        Content.append(df_slice['Concatenated'][index])
        Content_Vector.append(df_slice['content_vector'][index])
        
    json_data = df_slice.to_json()
    data_dict = json.loads(json_data) #200
    data_dict_list.append(data_dict)

In [ ]:
print(Content[0])

In [ ]:
df_slice.head(3)

In [ ]:
AZURE_SEARCH_INDEX_NAME=bid_name + "-test-index"
print(AZURE_SEARCH_INDEX_NAME)

search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)


In [ ]:
AZURE_SEARCH_INDEX_NAME 

In [ ]:
try:
    existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
except Exception as e:
    existing_index = None
    # The index doesn't exist, so create a new one
    
metadata = result_df.columns.tolist()
if existing_index == None:
    fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
    SearchField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),
    ]  

    
    result = create_index(AZURE_SEARCH_INDEX_NAME, fields)
    print(f' {result.name} created index') 
    
    
    ######
    dataframes_list = []
    data_dict_list = []
    partitionid = []
    #ID = []
    Content = []
    Content_Vector = []
    for i in range(200):
        start_row = i * rows_per_half
        end_row = (i + 1) * rows_per_half

        df_slice = df_concat.iloc[start_row:end_row] #slice df_concat
        #dataframes_list.append(df_slice)
        #convert DF to JSON
        partitionid.append(i)
        final_doc=[]
        for index in df_slice['ID'].index:
            Content = df_slice['Concatenated'][index]
            Content_Vector = df_slice['content_vector'][index]
            
            final_doc.append({'id': str(partitionid[i]), 
                              'content': Content,
                              'content_vector': Content_Vector,
                              'metadata': str(metadata),
                             })
        for batch in final_doc:
            try:
                result = search_client_v2.upload_documents(batch)
                print('documents uploaded')
            except Exception as e:
                print(f"Error uploading documents: {e}")
                print(repr(e))
                break

    print("Vector store added successfully")

    ######
                       

else:
    try:
        # Check if the index exists
        # existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
        # print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")
        ######
        dataframes_list = []
        data_dict_list = []
        partitionid = []
        #ID = []
        Content = []
        Content_Vector = []
        final_doc=[]
        for i in range(200):
            start_row = i * rows_per_half
            end_row = (i + 1) * rows_per_half
            df_slice = df_concat.iloc[start_row:end_row] #slice df_concat
            #dataframes_list.append(df_slice)
            #convert DF to JSON
            partitionid.append(i)
            sample_doc = []
            
            for index in df_slice['ID'].index:
                l.append(index)
                Content = df_slice['Concatenated'][index]
                Content_Vector = df_slice['content_vector'][index]
                sample_doc.append({'id': str(partitionid[i]), 
                                  'content': Content,
                                  'content_vector': Content_Vector,
                                  'metadata': str(metadata),
                                 })
            final_doc.append(sample_doc)

        for batch in final_doc:
            try:
                result = search_client_v2.upload_documents(batch)
                print('documents uploaded')
            except Exception as e:
                print(f"Error uploading documents: {e}")
                print(repr(e))
                break


        print("Vector store added successfully")

    except Exception as e:
        print(e)
        pass

In [ ]:
len(final_doc)

In [ ]:
sample_doc

VECTOR SEARCH SIMILARITY

In [ ]:
AZURE_SEARCH_INDEX_NAME= "vw-reportit-allreport-fast-index"
print(AZURE_SEARCH_INDEX_NAME)

search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)

In [ ]:
# Pure Vector Search
query = "show me pet related data"

embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 

query_embeddings = embeddings.embed_query(query)

vector = Vector(value= query_embeddings, k=1, fields="content_vector")
  
results = search_client_v2.search(  
    search_text=None,  
    vectors= [vector],
    select=["id" ,"content", "metadata"],
)  
  
for result in results:  
    print(f"id: {result['id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"content: {result['content']}") 
    print(f"metadata: {result['metadata']}\n") 
    break
    

In [ ]:
for i in df_concat['Concatenated']:
    print(i)
    break

NEW WORK

In [ ]:
try:
    existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
except Exception as e:
    existing_index = None
    # The index doesn't exist, so create a new one
    
metadata = result_df.columns.tolist()
if existing_index == None:
    fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="Division", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="EventDate", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="subject", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="description", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="Recommendations", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),
    ]

    result = create_index(AZURE_SEARCH_INDEX_NAME, fields)
    print(f' {result.name} created index')                        
    final_doc=[]
    for index in range(len(partitionid)):
        final_doc.append({'id': str(partitionid[index]), 
                          'Division': Division[index],
                          'EventDate': EventDate[index],  #df.iloc[index],
                          'subject': Subject[index],
                          'description': Description[index],
                          'Recommendations': Recommendations[index],
                          'content': str(data_dict_list[index]),
                          'metadata': str(metadata),
                         })

     
    # Define a list to store batches of documents
    #document_batches = [final_doc[i] for i in range(0, len(final_doc))]

    # Upload documents in batches
    for batch in final_doc:
        try:
            result = search_client_v2.upload_documents(batch)
            print('documents uploaded')
        except Exception as e:
            print(f"Error uploading documents: {e}")
            print(repr(e))
            break

    print("Vector store added successfully")

else:
    try:
        # Check if the index exists
        # existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
        # print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")
        final_doc=[]
        for index in range(len(partitionid)):
            final_doc.append({'id': str(partitionid[index]), 
                              'Division': Division[index],
                              'EventDate': EventDate[index],  #df.iloc[index],
                              'subject': Subject[index],
                              'description': Description[index],
                              'Recommendations': Recommendations[index],
                              'content': str(data_dict_list[index]),
                              'metadata': str(metadata),
                             })

        

        # Define a list to store batches of documents
        document_batches = [final_doc[i] for i in range(0, len(final_doc))]

        # Upload documents in batches
        for batch in document_batches:
            try:
                result = search_client_v2.upload_documents(batch)
                print('documents uploaded')
            except Exception as e:
                print(f"Error uploading documents: {e}") 
                print(repr(e))
                break

        print("Vector store added successfully")


    except Exception as e:
        print(e)
        pass